[![Open All Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/README.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python.git"

In [ ]:
import jsonfrom openalex_api import Configuration, ApiClient,AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApiconfiguration = Configuration(host="https://api.openalex.org")authors_api = AuthorsApi(ApiClient(configuration))concepts_api = ConceptsApi(ApiClient(configuration))funders_api = FundersApi(ApiClient(configuration))institutions_api = InstitutionsApi(ApiClient(configuration))publishers_api = PublishersApi(ApiClient(configuration))sources_api = SourcesApi(ApiClient(configuration))works_api = WorksApi(ApiClient(configuration))

# Get lists of entitiesIt's easy to get a list of entity objects from from the API:`/<entity_name>.` Here's an example:* Get a list of _all_ the concepts in OpenAlex:\  [`https://api.openalex.org/concepts`](https://api.openalex.org/concepts)

In [ ]:
response = concepts_api.get_concepts(	)print(json.dumps(response.to_dict(), indent=2))

This query returns a `meta` object with details about the query, a `results` list of [`Concept`](./../../api-entities/concepts/concept-object.ipynb) objects, and an empty [`group_by`](./../get-groups-of-entities.ipynb) list:```jsonmeta: {    count: 65073,    db_response_time_ms: 81,    page: 1,    per_page: 25    },results: [    // long list of Concept entities ],group_by: [] // empty```Listing entities is a lot more useful when you add parameters to [page](./paging.ipynb), [filter](./filter-entity-lists.ipynb), [search](./search-entities.ipynb), and [sort](./sort-entity-lists.ipynb) them. Keep reading to learn how to do that.